In [ ]:
import scipy.io
import scipy
import numpy as np
from sklearn.model_selection import train_test_split

import torch, torch.nn as nn
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter

from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
import torch.nn.functional as F

from sklearn.feature_selection import mutual_info_regression as MIR

In [ ]:
## Fixed params
eff = 0.5
nu = 1
PL_d0_dB = 31.67
alpha = 2
Z_stdDev = 2
W = 1e6
N0 = 1e-14
D = 50

scale_factor = 1e7
numChReal = 1000000
## params changing based on input file
P_AP = 2
p_max = 0.001

train_data_size = 100
num_of_user = 4

In [ ]:
input_data_folder='../input_data'
output_data_folder='../output/opt_DL_models/'
tensor_board_folder='../runs/optDL'

In [ ]:
def power_to_time_tensor(p_i, gh, gt, D, W, eff, P_AP, N0):
    t_i = D / (W * torch.log2(1 + (p_i * gt) / (N0 * W)))
    req_harv_energy = t_i * p_i
    t0 = torch.max(req_harv_energy / (eff * P_AP * gh), dim=1)[0]
    # tot_time=torch.sum(t_i,dim=1)+t0
    return scale_factor * torch.cat((t0.unsqueeze(dim=1), t_i), dim=1)

In [ ]:
def early_stopping(validation_loss_prev, validation_loss, counter,min_delta = 0.05):

    if (validation_loss - validation_loss_prev) < min_delta:
        counter +=1
    else:
        counter = 0
    return counter
      

In [ ]:
##  LOAD DATA
match = scipy.io.loadmat(input_data_folder + "/channel_gains.mat")
test_ind=np.load(input_data_folder+'/test_ind.npy')

In [ ]:
dnn_result = []
val_loss_list = []
# Load data
mat = scipy.io.loadmat(
    input_data_folder
    + "/result_compare_N014"
    + "_PAP"
    + str(P_AP)
    + "_M"
    + str(num_of_user)
    + "_pmax"
    + str(p_max)
    + ".mat"
)
outs = mat["tot_time_MRTTMA4"].T
outs = outs * scale_factor

gh = match["gh_arr"][0:num_of_user, 0:numChReal]
gt = match["gt_arr"][0:num_of_user, 0:numChReal]

gamma = gh * gt * eff * P_AP / (W * N0)
alpha = np.abs(scipy.special.lambertw(np.exp(-1) * (gamma - 1), k=0) + 1)

ins = np.concatenate((gh, gt, alpha, gamma)).T
indicies = np.arange(len(ins))

remaning_ind = list(set(indicies) - set(test_ind))

ins2 = ins[remaning_ind]
outs2 = outs[remaning_ind]

X_test = ins[test_ind]
y_test = outs[test_ind]
idx_test = list(test_ind)

X_train, X_val, y_train, y_val, idx_val, idx_test = train_test_split(
    ins2, outs2, remaning_ind, test_size=0.01, random_state=42
)

# Scale
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_val = ss.transform(X_val)
X_test = ss.transform(X_test)

# Feature Selection
mi = MIR(X_train[0:5000, :], np.sum(y_train[0:5000, :], 1))
feature_index = np.argsort(mi)
feat_size = 3 * num_of_user

feature_index = np.argsort(mi)
feature_index = feature_index[-1 * feat_size :]

## DNN Arch  ##
class OptNetMIR(nn.Module):

    def __init__(self):
        super(OptNetMIR, self).__init__()

        self.layer1 = nn.Linear(feat_size, 8 * num_of_user)
        self.layer2 = nn.Linear(8 * num_of_user, 8 * num_of_user)
        self.layer3 = nn.Linear(8 * num_of_user, 8 * num_of_user)
        self.layer4 = nn.Linear(8 * num_of_user, 4 * num_of_user)
        self.layer5 = nn.Linear(4 * num_of_user, 4 * num_of_user)

        self.output = nn.Linear(4 * num_of_user, num_of_user)

    def forward(self, input_x, input_x_inv):

        x1_1 = F.relu(self.layer1(input_x))
        x = F.relu(self.layer2(x1_1))
        x = F.relu(self.layer3(x))
        x = F.relu(self.layer4(x))
        x = F.relu(self.layer5(x))

        x = p_max * F.sigmoid(self.output(x))
        x2 = power_to_time_tensor(
            x,
            input_x_inv[:, 0:num_of_user],
            input_x_inv[:, num_of_user : 2 * num_of_user],
            D,
            W,
            eff,
            P_AP,
            N0,
        )

        return x2

# Dataset & DataLoder
torch_dataset = TensorDataset(
    torch.tensor(X_train[0:train_data_size, :].astype(np.float32)),
    torch.tensor(y_train[0:train_data_size, :].astype(np.float32)),
)
val_torch_dataset = TensorDataset(
    torch.tensor(X_val.astype(np.float32)), torch.tensor(y_val.astype(np.float32))
)

train_data_loader = DataLoader(torch_dataset, batch_size=32)
val_data_loader = DataLoader(val_torch_dataset, batch_size=1)

## TRAINING
# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter(
    tensor_board_folder
    + "/feat3N_trainsize"
    + str(train_data_size)
    + "sup_MIR_3N_feature"
    + "_M"
    + str(num_of_user)
    + "_PAP"
    + str(P_AP)
    + "_pmax"
    + str(p_max)
)
NO_EPOCHS = 20

val_loss_best = 10000000

model = OptNetMIR()
optimizer = Adam(model.parameters(), lr=1e-4)
scheduler = ReduceLROnPlateau(optimizer, "min")
loss_fn = nn.MSELoss()

for epoch_idx in range(NO_EPOCHS):
    model.train()
    epoch_loss = 0
    len_batches = 0
    for ii, sample in enumerate(tqdm(train_data_loader)):

        local_inp, local_tgt = sample
        local_inp_inv = torch.Tensor(ss.inverse_transform(local_inp))
        output = model(local_inp[:, feature_index], local_inp_inv)
        loss = loss_fn(output, local_tgt)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.detach().numpy()

    # Validation phase
    model.eval()
    val_loss = 0
    counter = 0
    with torch.no_grad():
        val_loss_prev=val_loss
        for jj, val_sample in enumerate(val_data_loader):
            local_inp, local_tgt = val_sample
            local_inp_inv = torch.Tensor(ss.inverse_transform(local_inp))

            output = model(local_inp[:, feature_index], local_inp_inv)
            val_loss += loss_fn(output, local_tgt)

        val_loss /= jj + 1

    if val_loss < val_loss_best:
        val_loss_best = val_loss
        # torch.save(model, output_data_folder+'/feat3N_model_N'+str(num_of_user)+'.pt')
        X_test_inverse = torch.Tensor(ss.inverse_transform(X_test))
        test_output = model(
            (torch.tensor(X_test[:, feature_index].astype(np.float32))),
            X_test_inverse,
        )
        if early_stopping(val_loss_prev, val_loss, counter) > 5:
            break
   

    # writer.add_scalar('val_loss',val_loss,epoch_idx)
    # writer.add_scalar('training loss', epoch_loss / (ii+1),epoch_idx)

val_loss_list.append(val_loss)
# np.save(output_data_folder+'/trainsize'+str(train_data_size)+'tot_time_DNN_N014'+'_PAP'+str(P_AP)+'_M'+str(num_of_user)+'_pmax'+str(p_max),(test_output/scale_factor).detach().numpy())
# dnn_result.append((test_output/scale_factor).sum(axis=1).mean().detach().item())
# np.save(output_data_folder+'/trainsize'+str(train_data_size)+'all_tot_times_N014'+'_PAP'+str(P_AP)+'_M'+str(num_of_user)+'_pmax'+str(p_max),dnn_result)

#np.save("drive/MyDrive/magazine_paper/val_loss_results_july_2024/expai", val_loss_list)

    